In [4]:
!pip install geopy
!pip install openpyxl
!pip install pandas
!pip install requests

   ---------------------------------------- 0.0/62.6 kB ? eta -:--:--
   ---------------------------------------- 62.6/62.6 kB 1.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/163.8 kB ? eta -:--:--
   ---------------------------------------- 163.8/163.8 kB 5.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/99.9 kB ? eta -:--:--
   ---------------------------------------- 99.9/99.9 kB 5.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/66.8 kB ? eta -:--:--
   ---------------------------------------- 66.8/66.8 kB 3.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/121.1 kB ? eta -:--:--
   ---------------------------------------- 121.1/121.1 kB 6.9 MB/s eta 0:00:00


In [10]:
import pandas as pd

df = pd.read_excel('Locations.xlsx')
df

df.to_csv('location.csv', index=False)

In [ ]:
import pandas as pd
from geopy.geocoders import Nominatim

# Load the Excel file
df = pd.read_excel('combined.xlsx')

# Extract address, postal code, city, and country information
addresses = df['Adresse'] + ', ' + df['Postleitzahl'] + ', ' + df['Ort'] + ', ' + df['Land']

# Initialize the Nominatim geolocator
geolocator = Nominatim(user_agent="myGeocoder")

# Create empty lists to store latitude and longitude
latitudes = []
longitudes = []

# Loop through each address and retrieve latitude and longitude
for address in addresses:
    try:
        location = geolocator.geocode(address)
        latitudes.append(location.latitude)
        longitudes.append(location.longitude)
    except:
        latitudes.append(None)
        longitudes.append(None)

# Create new columns in the DataFrame to store latitude and longitude
df['Latitude'] = latitudes
df['Longitude'] = longitudes

In [14]:
import pandas as pd
import requests
from requests.structures import CaseInsensitiveDict

# Replace 'YOUR_API_KEY' with your actual Geoapify API key
api_key = 'edb8cde1acdc48f39b75476ff37d05f3'

# Load the Excel file
df = pd.read_excel(r'C:\Users\André\Desktop\Universität\Universität\Master\Masterarbeit\webcrawler\data\poland_members.xlsx')

# Define the API endpoint
base_url = 'https://api.geoapify.com/v1/geocode/search'

# Define the headers for the API request
headers = CaseInsensitiveDict()
headers['Accept'] = 'application/json'

# Initialize empty lists to store latitude and longitude
latitudes = []
longitudes = []

# Loop through each address to retrieve latitude and longitude
for index, row in df.iterrows():
    address = row['Full Address']

    if address == "nan, nan, nan":
        latitudes.append(None)
        longitudes.append(None)
        continue

    # Define the parameters for the API request
    params = {
        'text': address,
        'apiKey': api_key,
        'lang': 'en',  # Set the language to English
        'limit': 1,  # Set the limit to 1 to get the first result
        'type': 'amenity'  # Set the location type to amenity (you can adjust this based on your needs)
    }

    # Send the API request
    response = requests.get(base_url, params=params, headers=headers)
        # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the JSON response
        response_data = response.json()

        # Check if 'features' list is not empty
        if len(response_data['features']) > 0:
            # Extract latitude and longitude from the response
            latitude = response_data['features'][0]['geometry']['coordinates'][1]
            longitude = response_data['features'][0]['geometry']['coordinates'][0]

            # Append latitude and longitude to the lists
            latitudes.append(latitude)
            longitudes.append(longitude)

            # Print the address and latitude/longitude for each successful request
            print(f"Address: {address}")
            print(f"Latitude: {latitude}")
            print(f"Longitude: {longitude}")
            print()
        else:
            print(f"No results found for address: {address}")
            latitudes.append(None)
            longitudes.append(None)
    else:
        print(f"Error geocoding address: {address}")
        latitudes.append(None)
        longitudes.append(None)

# Create new columns in the DataFrame to store latitude and longitude
df['Latitude'] = latitudes
df['Longitude'] = longitudes


Address: al. Powsta?ców Warszawy 12, 35-959 Rzeszów
Latitude: 50.019363
Longitude: 21.98884

Address: Krakowskie Przedmie?cie 26/28, 00-927 Warszawa
Latitude: 52.240883
Longitude: 21.017806

Address: Narutowicza 11/12, 80-233 Gda?sk
Latitude: 48.18856565
Longitude: 17.18911517577356

Address: Akademicka 2a, 44-100 Gliwice
Latitude: 50.2895655
Longitude: 18.677579

Address: al. Mickiewicza 30, 30-059 Kraków
Latitude: 50.064485
Longitude: 19.923628

Address: Warszawska 24, 31-155 Kraków
Latitude: 50.0712118
Longitude: 19.9431226

Address: Pogodna 3, 05-083 Wiktorów
Latitude: 52.262183
Longitude: 20.692474

Address: Plac Politechniki 1, 00-661 Warszawa
Latitude: 52.222213
Longitude: 21.0066552

Address: Cieplaka 1c, 41-300 D?browa Górnicza
Latitude: 50.326453
Longitude: 19.191737

Address: Al. Niepodleg?o?ci 34, 61-714 Pozna?
Latitude: 48.337925
Longitude: 15.921685

Address: Bankowa 12, 40-007 Katowice
Latitude: 50.2608163
Longitude: 19.0285342

Address: Podwale 17, 00-252 Warszawa
Latit

In [15]:
df = df.drop(['Unnamed: 4'], axis=1)
print(df)
df.to_csv('locations/location_poland.csv', index=False)

    Unnamed: 0                                               Name   Category  \
0            0                   Rzeszow University of Technology  Education   
1            1                               University of Warsaw  Education   
2            2                    Gda?sk University of Technology  Education   
3            3                  Silesian University of Technology  Education   
4            4                           AGH University of Krakow  Education   
..         ...                                                ...        ...   
92          92                                 AUTOSAN SP. Z O.O.  Transport   
93          93                         Hydrogen Refueling Station  Transport   
94          94                 Hydrogen Refueling Station - ORLEN  Transport   
95          95  Centralny Klaster Wodorowy im. Braci ?aszczy?s...  Transport   
96          96                         Hydrogen Refueling Station  Transport   

                           Street      

In [81]:
df1 = pd.read_excel('data-MDu07.xlsx')
df2 = pd.read_excel('data-krTLV.xlsx')

In [82]:
df2['lat'] = df2['LAT']
df2['lon'] = df2['LON']
df1['lat'] = df1['Lat']
df1['lon'] = df1['Lon']

In [83]:
df1.drop(['Lat', 'Lon'], axis=1)

,Title,Partner,Partner ohne Markdown,Website,Straße,PLZ,Ort,Land,Partner aus…,Beteiligt in…,Funktion als…,Webadresse,Column 1,lat,lon
0,NaN,[ABB AG](https://www.abb.de/),ABB AG,https://www.abb.de/,Kallstadter Str. 1,68309,Mannheim,Baden-Württemberg,Wirtschaft,FertiRob,Partner,www.abb.de,NaN,49.489291,8.4673098
1,NaN,[Advanced Training Technologies GmbH](https://...,Advanced Training Technologies GmbH,https://sk-att.com/,Am Förderturm 5 c,44575,Castrop-Rauxel,Nordrhein-Westfalen,Wirtschaft,HyPLANT100,Partner,sk-att.com,NaN,51.564619,7.3106175
2,NaN,[Alantum Europe GmbH](https://www.alantum.com/),Alantum Europe GmbH,https://www.alantum.com/,Balanstr. 73 / Geb. 21A,81541,München,Bayern,Wirtschaft,AEL4GW,Partner,www.alantum.com,NaN,48.259248,11.4354419
3,NaN,[Albert-Ludwigs-Universität Freiburg](https://...,Albert-Ludwigs-Universität Freiburg,https://www.uni-freiburg.de/,Georges-Köhler-Allee 103,79110,Freiburg im Breisgau,Baden-Württemberg,Wissenschaft,Fluorfreie MEA,Partner,www.uni-freiburg.de,NaN,47.965146,7.9573306
4,NaN,[AUDI Aktiengesellschaft](https://www.audi.de/),AUDI Aktiengesellschaft,https://www.audi.de/,Auto-Union-Str. 1,85057,Ingolstadt,Bayern,Wirtschaft,PEP.IN,Partner,www.audi.de,NaN,48.763016,11.4250395
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,NaN,[Zentrum für Brennstoffzellen-Technik GmbH](ht...,Zentrum für Brennstoffzellen-Technik GmbH,https://www.zbt.de/,Carl-Benz-Str. 201,47057,Duisburg,Nordrhein-Westfalen,Wissenschaft,"AlFaKat, Degrad-EL3, PEP.IN",Koordinator und Partner,www.zbt.de,NaN,51.333120,6.5623343
113,NaN,[Öko-Institut. Institut für angewandte Ökologi...,Öko-Institut. Institut für angewandte Ökologie...,https://www.oeko.de/,Rheinstr. 95,64295,Darmstadt,Hessen,Wissenschaft,ReNaRe,Partner,www.oeko.de,NaN,49.872775,8.651177
114,NaN,[Semodia GmbH](https://www.semodia.com),Semodia GmbH,https://www.semodia.com,Meißner Str. 37,1445,Radebeul,Sachsen,Wirtschaft,eModule,Partner,www.semodia.com,NaN,51.097174,13.6923622
115,NaN,[Semodia GmbH](https://www.semodia.com),Semodia GmbH,https://www.semodia.com,Meißner Str. 37,1445,Radebeul,Sachsen,Wirtschaft,eModule,Partner,www.semodia.com,NaN,52.194911,11.5985197


In [84]:
df2.drop(['LAT', 'LON'], axis=1)

,Title,Partner,Partner ohne Markdown,Website,Straße,PLZ,Ort,Land,Partner aus…,Beteiligt in…,Funktion als…,Webadresse,Mehrere Partner,lat,lon
0,NaN,[ADLARES GmbH](www.adlares.com),ADLARES GmbH,https://www.adlares.com/,Oderstr. 65,14513,Teltow,Brandenburg,Wirtschaft,GetH2,Partner,www.adlares.com,NaN,52.382685,13.276558
1,NaN,[Agentur für Erneuerbare Energien e.V.](www.un...,Agentur für Erneuerbare Energien e.V.,https://www.unendlich-viel-energie.de/,EUREF-Campus 16,10829,Berlin,Berlin,Wirtschaft,Campfire,Partner,www.unendlich-viel-energie.de,NaN,52.476629,13.356394
2,NaN,[BASF SE - Public Funding Europe RC/OFP - C006...,BASF SE - Public Funding Europe RC/OFP - C006,https://www.basf.com/,Carl-Bosch-Str. 38 / Tor 7,67063,Ludwigshafen am Rhein,Rheinland-Pfalz,Wirtschaft,Derivate,Partner,www.basf.com,NaN,49.513021,8.421946
3,NaN,[Brandenburgische Technische Universität Cottb...,Brandenburgische Technische Universität Cottbu...,https://www.b-tu.de/,Siemens-Halske-Ring 13,3046,Cottbus,Brandenburg,Wissenschaft,"Systemanalyse, Mukran",Partner,www.b-tu.de,NaN,51.696222,9.403191
4,NaN,[Bundesanstalt für Materialforschung und -prüf...,Bundesanstalt für Materialforschung und -prüfu...,https://www.bam.de/,Unter den Eichen 87,12205,Berlin,Berlin,Wirtschaft,Normung,Partner,www.bam.de,NaN,52.433040,13.294795
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,NaN,[VDEh-Betriebsforschungsinstitut Gesellschaft ...,VDEh-Betriebsforschungsinstitut Gesellschaft m...,https://www.bfi.de/,Sohnstr. 69,40237,Düsseldorf,Nordrhein-Westfalen,Wirtschaft,Systemanalyse,Partner,www.bfi.de,NaN,51.238035,6.810505
79,NaN,[Vision Electric Super Conductors GmbH](www.ve...,Vision Electric Super Conductors GmbH,https://www.vesc-superbar.de/,Morlauterer Str. 21,67657,Kaiserslautern,Rheinland-Pfalz,Wirtschaft,Flüssig Wasserstoff,Partner,www.vesc-superbar.de,NaN,49.452727,7.802779
80,NaN,[VNG AG](www.vng.de),VNG AG,https://www.vng.de/,Braunstr. 7,4347,Leipzig,Sachsen,Wirtschaft,Systemanalyse,Partner,www.vng.de,NaN,51.362600,12.434246
81,NaN,[Zentrum für Brennstoffzellen-Technik GmbH](ww...,Zentrum für Brennstoffzellen-Technik GmbH,https://www.zbt.de/,Carl-Benz-Str. 201,47057,Duisburg,Nordrhein-Westfalen,Wissenschaft,Campfire,Partner,www.zbt.de,NaN,51.415235,6.794585


In [85]:

frames = [df1, df2]
merge_df = pd.concat(frames)

merge_df
merge_df.to_csv('deutschland_projecte.csv', index=False)

In [86]:
# Filter the DataFrame to show rows where the 'Partner' column contains NaN values
nan_rows = df1[df1['Partner'].isnull()]

# Print the filtered DataFrame
print(nan_rows)

Empty DataFrame
Columns: [Lat, Lon, Title, Partner, Partner ohne Markdown, Website, Straße, PLZ, Ort, Land, Partner aus…, Beteiligt in…, Funktion als…, Webadresse, Column 1, lat, lon]
Index: []


In [78]:
df1.loc[12,'Lat']

'50.1427962,9.0464884,,De Nora Deutschland GmbH,De Nora Deutschland GmbH,https://www.denora.com,Industriestr. 17,63517,Rodenbach,Hessen,Wirtschaft,"INSTALL-AWE, PrometH2eus",Partner,www.denora.com,'